In [1]:
from telnetlib import X3PAD
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

from sklearn.utils import shuffle
import pickle 
import sqlite3


In [2]:
df_2020 = pd.read_csv('merged_gw2020.csv',low_memory = False)
df_2021 = pd.read_csv('merged_gw2021.csv',low_memory = False)


In [3]:
season_2020 = pd.DataFrame({'season' : ['2019/20']*df_2020.shape[0]   })
season_2021 = pd.DataFrame({'season' : ['2020/21']*df_2021.shape[0]   })

In [4]:
df_2020 =pd.concat([df_2020, season_2020] ,axis = 'columns')
df_2021 =pd.concat([df_2021, season_2021] ,axis = 'columns')


In [5]:
df= pd.concat([df_2020 , df_2021], axis = 'index',ignore_index = True)


In [6]:
df["was_home"] = df["was_home"].astype(int)

In [7]:
df.sort_values(by=['name','season','GW'], inplace =True)
df.reset_index(inplace = True)


df.drop(['index'],axis= 'columns' , inplace =True)

In [8]:
df['id'] = df.groupby(['name']).ngroup()

In [9]:
df['pos']= df.groupby(['position']).ngroup()

In [10]:
df['teams'] = df.groupby(['team']).ngroup()

In [11]:
identity = df['id'].unique()

In [12]:

df_id = df.loc[: , ['id', 'name' ,'position']]
df_id.reset_index(inplace = True)
df_id.drop_duplicates(subset='id',keep = 'first',inplace = True)

df_id.drop(['index'],axis= 'columns' , inplace =True)

In [13]:
x = 25

In [14]:
y=x
Gameweeks=[]

In [15]:
for i in range(5):
    Gameweeks.append(y)
    y=y-1


In [16]:
dataframes = {}

In [17]:
for GW in Gameweeks:
    filt=(df['GW'] == GW) & (df['season'] == '2020/21')
    dataframes[GW] = df[filt]
    dataframes[GW].reset_index(inplace = True)
    dataframes[GW].drop(['index'],axis= 'columns' , inplace =True)
    dataframes[GW].drop([ 'kickoff_time','season', 'name' , 'position','pos' ,'GW' ,'team','teams'],axis= 'columns' , inplace =True)
    

/home/aavash/Downloads/jupyter/environment/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
n = 0
for keys in dataframes:
    column = []
    
    for value in dataframes[keys].columns:
        
        if (value != 'id')  & (value!='team'):
            value = value +f'_{n}' 
        column.append(value)
        
    dataframes[keys].columns = column
    n=n-1   

In [ ]:
for keys in dataframes:
    dataframes[keys].drop_duplicates(subset='id' , keep ='first' , inplace = True)
    dataframes[keys].reset_index(inplace = True)
    dataframes[keys].drop(['index'],axis= 'columns' , inplace =True)

In [20]:
df1 = pd.merge(dataframes[x] , dataframes[x-1], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-2], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-3], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-4], on = 'id' , how = 'outer')

In [21]:
filt = (df['GW'] == x) | (df['GW'] == x-1) | (df['GW'] == x-2) | (df['GW'] == x-3) | (df['GW'] == x-4)
df_position = df.loc[filt , ['id' , 'pos','teams']]

In [22]:
df_position.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_position.reset_index(inplace = True)
df_position.drop(['index'],axis= 'columns' , inplace =True)

In [23]:
df1.drop_duplicates(subset='id',keep = 'first',inplace = True)
df1.reset_index(inplace = True)
df1.drop(['index'],axis= 'columns' , inplace =True)

In [24]:
df1 = df1.fillna(df1.median())

In [25]:
df1

,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,element_0,fixture_0,goals_conceded_0,goals_scored_0,...,team_a_score_-4,team_h_score_-4,threat_-4,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4
0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,250.0,0.0,0.0,...,3.0,2.0,0.0,0.0,-388.0,490.0,878.0,53.0,0.0,0.0
1,3.8,0.0,0.0,0.0,0.0,3.3,411.0,245.0,2.0,0.0,...,3.0,2.0,0.0,0.0,-42234.0,1424.0,43658.0,53.0,0.0,0.0
2,0.7,0.0,0.0,13.0,0.0,25.3,575.0,242.0,1.0,0.0,...,1.0,3.0,2.0,2.0,911.0,1204.0,293.0,49.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,496.0,247.0,0.0,0.0,...,1.0,2.0,0.0,0.0,119.0,1499.0,1380.0,44.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,288.0,246.0,0.0,0.0,...,1.0,0.0,2.0,2.0,1386.0,6249.0,4863.0,51.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,0.3,0.0,0.0,0.0,0.0,0.0,367.5,246.0,0.0,0.0,...,2.0,2.0,0.0,0.0,-131.0,55.0,186.0,50.0,1.0,0.0
702,0.3,0.0,0.0,0.0,0.0,0.0,367.5,246.0,0.0,0.0,...,2.0,2.0,0.0,0.0,-36.0,0.0,36.0,65.0,1.0,0.0
703,0.3,0.0,0.0,0.0,0.0,0.0,367.5,246.0,0.0,0.0,...,2.0,2.0,0.0,1.0,-79499.0,818.0,80317.0,56.0,1.0,0.0
704,0.3,0.0,0.0,0.0,0.0,0.0,367.5,246.0,0.0,0.0,...,2.0,2.0,0.0,0.0,-3263.0,227.0,3490.0,86.0,1.0,0.0


,id,pos,teams
0,0,1,3
1,1,0,21
2,2,3,4
3,3,3,3
4,4,2,16
...,...,...,...
933,960,3,16
934,961,0,8
935,962,2,2
936,963,0,10


In [27]:
df1 = pd.merge(df1 , df_position, on = 'id' , how = 'inner')

In [28]:
columns = df1.columns
remove = ('pos','teams','id','total_points')
a = set(columns)
b = a.difference(remove)
columns = list(b)

In [29]:
df1.loc[:,columns] = StandardScaler().fit_transform(df1.loc[:, columns])

In [30]:
X=df1.drop(['id'  ,'pos','teams'],axis ='columns') 


In [31]:
import pickle

with open('model_pickle','rb') as file:
    mp = pickle.load(file)

In [32]:
df1['predict'] = mp.predict(X)




In [33]:
df1 = pd.merge(df1 , df_id, on = 'id' , how = 'inner')

In [34]:
df_op = df1.loc[:,['predict' , 'name' , 'position']]

In [35]:
df_op.isnull().values.any()

False

In [37]:
df_op['predict'].max()

5.46

In [ ]:
df_op.to_csv('final_result')